# My IMDb Ratings

In [ ]:
import os
import numpy as np
import pandas as pd
import altair as alt

path = os.path.expanduser('../data/imdb.csv')

In [ ]:
df = pd.read_csv(path, header=None,
                 names=['title', 'release_year', 'genre', 'rating', 'user_rating', 'votes', 'range_year'])
df['release_year'] = df.release_year.astype('category')

## Count by year

In [ ]:
tmp = df.groupby('release_year')['release_year'].count()
data = pd.DataFrame({'release_year': tmp.index, 'count': tmp.values})

In [ ]:
chart = alt.Chart(data).mark_bar().encode(
  x=alt.X('release_year', title='Release Year'),
  y=alt.Y('count', title='Count'))
text = chart.mark_text(dy=-10).encode(text='count')

chart + text

## Rating per year

In [ ]:
data = pd.DataFrame({'release_year': df.release_year, 'rating': df.rating, 'type': 'POP'})
data = data.append(pd.DataFrame({'release_year': df.release_year, 'rating': df.user_rating, 'type': 'USR'}))
data['type'] = data.type.astype('category')

In [ ]:
chart1 = alt.Chart(data[(data['release_year'].astype('int') >= 1990) & (data['release_year'].astype('int') <= 2000)]).mark_boxplot().encode(
  x='type',
  y='rating',
  column='release_year',
  color='type')

chart2 = alt.Chart(data[(data['release_year'].astype('int') >= 2001) & (data['release_year'].astype('int') <= 2010)]).mark_boxplot().encode(
  x='type',
  y='rating',
  column='release_year',
  color='type')

chart3 = alt.Chart(data[data['release_year'].astype('int') > 2010]).mark_boxplot().encode(
  x='type',
  y='rating',
  column='release_year',
  color='type')

chart1 & chart2 & chart3

## Votes

### Distribution of votes

In [ ]:
chart = alt.Chart(df).mark_bar().encode(
  x=alt.X('votes:Q', bin=alt.Bin(maxbins=50), axis=alt.Axis(title='Total Votes', labelAngle=-90)),
  y=alt.Y('count()', title='Bin Count'))

text = chart.mark_text(dy=-10).encode(text='count()')

(chart + text).properties(width=875)

### Distribution of votes by year

In [ ]:
alt.Chart(df[df['release_year'].astype('int') > 1980]).mark_boxplot().encode(
  x=alt.X('release_year', title='Release Year'),
  y=alt.Y('votes', title='Number of votes'))

## Ratings distribution by genre

In [ ]:
data = pd.DataFrame({'release_year': df.release_year, 'rating': df.rating, 'type': 'POP'})
data = data.append(pd.DataFrame({'release_year': df.release_year, 'rating': df.user_rating, 'type': 'USR'}))
data['type'] = data.type.astype('category')

all_genres = set()
for gl in df['genre'].apply(lambda x: x.split(',')).tolist():
   all_genres.update([g.strip() for g in gl])

for g in all_genres:
    flag = df['genre'].apply(lambda x: int(g in x))
    flag = flag.append(flag)
    data[f'g_{g}'] = flag

In [ ]:
charts = [
    alt.Chart(data[(data[f'g_{g}'] == 1)]).mark_boxplot().encode(
      x='type',
      y='rating',
      color='type'
    ).properties(
        title=g
    )
    for g in all_genres
]

final_chart = None
for i in range(5):
    c = charts[i * 5]
    for j in range(1, 5):
        c |= charts[i * 5 + j]

    if final_chart is None:
        final_chart = c
    else:
        final_chart &= c

final_chart